In [1]:
from keras.models import load_model
from Class_Image import Image
from os.path import abspath
from Class_training import *
from Model_comparisons import *
from os.path import abspath, join
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG19
import keras
from tensorflow.keras import Input, Model
%matplotlib inline

In [2]:
print("Loading data...")

DATA_FOLDER ="Z:\Comparisons_npy"#"D:\\Ottawa\\Comparisons_npy"
folder_path = abspath("Examples/Training_results/Simple")

data_left = np.load(join(DATA_FOLDER, "train", "train_left_224.npy"))
data_right = np.load(join(DATA_FOLDER, "train", "train_right_224.npy"))
data_label = np.load(join(DATA_FOLDER, "train", "train_labels_224.npy"))
labels_score = np.load(join(DATA_FOLDER, "train", "train_labels_score_224.npy"))
train_data = [data_left, data_right]
print("Done")
#simple_training(left, right, labels, comparisons_model, [224], TRAIN_RESULTS, 0.2, 400, 6)

Loading data...
Done


In [2]:
def batched_forward_pass(data, model, batch_size=20):
    len_data = data.shape[0]
    a = model(data[0:batch_size])
    
    for i in range(batch_size, len_data, batch_size):
        v = model(data[i:i+batch_size])
        a = np.concatenate((a, v), 0)
        
    return a

def simple_training( DATA_FOLDER,
                    model_function, model_function_args, folder_path, val_split, epochs, batch_size):
    """
    Train a siamese model and store results in a folder.

    :param data_left: left images array
    :type data_left: np.array
    :param data_right: right images array
    :type data_right: np.array
    :param data_label: labels array
    :type data_label: np.array
    :param model_function: function which build the model to train with k fold
    :type model_function: function
    :param model_function_args: arguments of the model building function
    :type model_function_args: list
    :param folder_path: path of the folder where results are stored
    :type folder_path: str
    :param val_split: set proportion dedicated to the validation set
    :type val_split: float
    :param epochs: number of training epochs
    :type epochs: int
    :param batch_size: batch size
    :type batch_size: int
    :return:
    :rtype:
    """
    # Create folder to store results
    folder_path = safe_folder_creation(folder_path)

    # Build model
    conv_model = model_function(*model_function_args)
    #train_data = [data_left, data_right]
    
    vgg_trainable = 'block3_pool'
    vgg_include_until = 'block4_pool'
    
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg_no_train = Model(inputs=vgg.input, outputs=vgg.get_layer(vgg_trainable).output)
    
    try:
        embeddings_left = np.load(folder_path + "/embeddings_left_{}.npy".format(vgg_trainable))
        embeddings_right = np.load(folder_path + "/embeddings_right_{}.npy".format(vgg_trainable))
        print("Embeddings successfully loaded from files")
        
    except FileNotFoundError:
        print("Creating and saving embeddings...")
        data_left = np.load(join(DATA_FOLDER, "train", "train_left_224.npy"))
        embeddings_left = batched_forward_pass(data_left, vgg_no_train, 2)
        del data_left
        print("Left complete")
        
        data_right = np.load(join(DATA_FOLDER, "train", "train_right_224.npy"))
        embeddings_right = batched_forward_pass(data_right, vgg_no_train, 2)
        del data_right
        print("Right Complete")
        
        np.save(os.path.join(folder_path, "/embeddings_left_{}.npy".format(vgg_trainable)), embeddings_left)
        np.save(os.path.join(folder_path, "/embeddings_right_{}.npy".format(vgg_trainable)), embeddings_right)
        
    return
    train_data[embeddings_left, embeddings_right]
    
    data_label = np.load(join(DATA_FOLDER, "train", "train_labels_224.npy"))
    # Train model
    model_fitted = conv_model.fit(train_data, data_label, vgg_trainable=vgg_trainable,
                                  epochs=epochs, batch_size=batch_size, validation_split=val_split)

    # Save model and plots
    model_path = os.path.join(folder_path, 'fitted_model.h5')
    conv_model.save(model_path)
    plot_validation_info(model_fitted, folder_path)

    # Save also weights and structure for backup
    weights_path = os.path.join(folder_path, 'weights.h5')
    conv_model.save_weights(weights_path)
    json_path = os.path.join(folder_path, 'structure.json')
    save_structure_in_json(conv_model, json_path)

simple_training("Z:\Comparisons_npy", comparisons_model, [224], "Examples/Training_results/Simple", 0.2, 5, 8)


Folder already exists : Examples/Training_results/Simple
 Please enter a new path !vfgbcxnjcghm
Creating and saving embeddings...


MemoryError: Unable to allocate 1.78 GiB for an array with shape (2386, 28, 28, 256) and data type float32

In [ ]:
batched_forward_pas

In [ ]:

# Build model
conv_model = comparisons_model(224)


In [ ]:
# Train model
#model_fitted = conv_model.fit(train_data, data_label,epochs=400, batch_size=6, validation_split=0.2)



In [ ]:
plot_validation_info(conv_model,folder_path)

In [ ]:
conv_model.history

In [ ]:

# Save model and plots
folder_path = safe_folder_creation(folder_path)

model_path = os.path.join(folder_path, 'fitted_model.h5')
conv_model.save(model_path)
plot_validation_info(model_fitted, folder_path)

# Save also weights and structure for backup
weights_path = os.path.join(folder_path, 'weights.h5')
conv_model.save_weights(weights_path)
json_path = os.path.join(folder_path, 'structure.json')
save_structure_in_json(conv_model, json_path)


In [ ]:
from tensorflow.keras.applications import VGG19
img_size=224
vgg_feature_extractor = VGG19(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

for layer in vgg_feature_extractor.layers[:-4]:
        layer.trainable = False

In [ ]:
vgg_feature_extractor.summary()

In [ ]:
conv_model([np.ones([1,224,224,3])] * 2)

In [ ]:
model.weights[-1]

In [ ]:
vgg_feature_extractor.weights[-1]

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow.keras.backend as K

K._get_available_gpus()

In [ ]:
plt.imshow(left[0]/255)

In [ ]:
vgg_feature_extractor(np.expand_dims(left[0],0))

In [ ]:
model(np.expand_dims(left[0],0))

In [ ]:
np.expand_dims(left[0],0)